## Summary notes

This was the initial **#TidyTuesday** project, posted back on April 2nd, 2018.
Here's the motivating tweet from [@Thomas_Mock](https://twitter.com/thomas_mock):

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">This week we will be exploring average US tuition costs! <br><br>The dataset is pre-cleaned but not fully tidy, so do your best to tidy it up and make an interesting (but quick!) plot! <br><br>The full rules, data, and more will always be at <a href="https://t.co/8NaXR93uIX">https://t.co/8NaXR93uIX</a></p>&mdash; Tom Mock ❤️ Quarto (@thomas_mock) <a href="https://twitter.com/thomas_mock/status/980921605407768577?ref_src=twsrc%5Etfw">April 2, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

Source data was taken from [http://trends.collegeboard.org](http://trends.collegeboard.org).
Further data on the US states was taken from the [American National Standards Institute](https://www.ansi.org/) (ANSI).
The ANSI data was used to bring in the state ID[^1] and the state abbreviations.

The tuition fee data was tidy, but there were some irritating formatting errors in the column titles.
These were cleaned up in the tidying pipeline with the help of several functions.
The ANSI data also needed some minor cleaning of the column titles so they were in snake case.
We did this with a dictionary that mapped the title case to snake cases.

We rescaled the percentage change data so that it was consistent with the example plot.

Two visualisations were produced:
The first is a **chloropleth heatmap** of the percentage change in fees from 2010 to 2015;
and the second is a **simple horiziontal bar chart** of the cost of tuition in 2015.

## Dependencies

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data as vdata

## Constants

In [2]:
FEE_URL = ('https://github.com/rfordatascience/tidytuesday/blob/master/'
           + 'data/2018/2018-04-02/us_avg_tuition.xlsx?raw=true')

In [3]:
ANSI_URL = 'https://www2.census.gov/geo/docs/reference/state.txt'

## Main

### Load the data

In [7]:
fee = pd.read_excel(FEE_URL)
fee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   State       50 non-null     object 
 1   2004-05     50 non-null     float64
 2   2005-06     50 non-null     float64
 3   2006-07     50 non-null     float64
 4     2007-08   50 non-null     float64
 5   2008-09     50 non-null     float64
 6   2009-10     50 non-null     float64
 7   2010-11     50 non-null     float64
 8   2011-12     50 non-null     float64
 9   2012-13     50 non-null     float64
 10  2013-14     50 non-null     float64
 11  2014-15     50 non-null     float64
 12  2015-16     50 non-null     float64
dtypes: float64(12), object(1)
memory usage: 5.2+ KB


In [8]:
ansi = pd.read_csv(ANSI_URL, sep='|')
ansi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STATE       57 non-null     int64 
 1   STUSAB      57 non-null     object
 2   STATE_NAME  57 non-null     object
 3   STATENS     57 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.9+ KB


In [9]:
states = alt.topo_feature(vdata.us_10m.url, 'states')

### Prepare the data

In [10]:
v_fee = fee.melt(
    id_vars='State', var_name='year', value_name='fee'
).rename(
    columns={'State': 'state_name'}
)
v_fee['year'] = v_fee['year'].str[:-3]
v_fee['year'] = v_fee['year'].str.strip()
v_fee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state_name  600 non-null    object 
 1   year        600 non-null    object 
 2   fee         600 non-null    float64
dtypes: float64(1), object(2)
memory usage: 14.2+ KB


In [11]:
v_ansi = ansi.rename(
    columns={
        'STATE': 'id',
        'STUSAB': 'state_abbr',
        'STATE_NAME': 'state_name'
    }
)
v_ansi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          57 non-null     int64 
 1   state_abbr  57 non-null     object
 2   state_name  57 non-null     object
 3   STATENS     57 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.9+ KB


### Visualise the data

Percentage Change in State Average Annual Tuition Fees (2010-2015)

In [12]:
_gsource = (
    v_fee.query("year in ['2010', '2015']")
    .groupby('state_name')['fee']
    .pct_change()
    .dropna()
    .set_axis(v_fee['state_name'].drop_duplicates())
    .rename('diff (%)')
    .mul(100)
    .astype('int')
    .to_frame()
    .merge(v_ansi, on='state_name')
)

alt.Chart(_gsource).mark_geoshape(
    stroke='black'  # add state borders
).encode(
    shape='geo:G',
    color=alt.Color("diff (%)", scale=alt.Scale(scheme="blues")),
    tooltip=['state_abbr', 'diff (%)']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=states, key='id'),
    as_='geo'
).project(
    type='albersUsa'
).properties(
    title=('Percentage Change in State Average Annual Tuition Fees'
           + ' (2010-2015)'),
    width=800,
    height=600
).configure_title(
    fontSize=16,
    anchor='start'
)

alt.Chart(...)

Average State Tuition Fee in the USA in 2015.

In [13]:
_gsource = v_fee.query("year == '2015'")

alt.Chart(_gsource).mark_bar().encode(
    x=alt.X('fee', title='fee (USD)'),
    y=alt.Y('state_name', title='state', sort='-x')
).properties(
    title=('Average State Tuition Fee in the USA in 2015'),
    width=600,
    height=800
).configure_title(
    fontSize=16,
    anchor='start'
)

alt.Chart(...)

[^1]: Used for mapping the data on a chloropleth map.